In [2]:
!pip install langchain
!pip install langchain_community
!pip install PyPDF
!pip install ctransformers
!pip install faiss-cpu
 

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
 
loader = PyPDFLoader("law1.pdf")
 
pages = loader.load_and_split()
 
text_splitter = RecursiveCharacterTextSplitter(  
    separators = ["\n \n", "\n"],    
    chunk_size = 400,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex = False,
)
texts = text_splitter.split_documents(pages)
 
texts[1]


Document(metadata={'source': 'law1.pdf', 'page': 0}, page_content='제2조(신의성실) ①권리의 행사와 의무의 이행은 신의에 좇아 성실히 하여야 한다.\n②권리는 남용하지 못한다.\n \n       제2장 인\n         제1절 능력\n \n제3조(권리능력의 존속기간) 사람은 생존한 동안 권리와 의무의 주체가 된다.\n \n제4조(성년) 사람은 19세로 성년에 이르게 된다.\n[전문개정 2011. 3. 7.]\n \n제5조(미성년자의 능력) ①미성년자가 법률행위를 함에는 법정대리인의 동의를 얻어야 한다. 그러나 권리만을 얻거나\n의무만을 면하는 행위는 그러하지 아니하다.\n②전항의 규정에 위반한 행위는 취소할 수 있다.\n \n제6조(처분을 허락한 재산) 법정대리인이 범위를 정하여 처분을 허락한 재산은 미성년자가 임의로 처분할 수 있다.')

In [4]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")

C:\Users\AISW-203-102\AppData\Local\Temp\ipykernel_21520\2814393515.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")
c:\Users\AISW-203-102\study\myenv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\AISW-203-102\study\myenv\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. 

In [5]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embedding_function)

In [6]:
query="신의성실"
docs =db.similarity_search(query)
#print(docs[0].page_content)
docs

[Document(metadata={'source': 'law1.pdf', 'page': 0}, page_content='제2조(신의성실) ①권리의 행사와 의무의 이행은 신의에 좇아 성실히 하여야 한다.\n②권리는 남용하지 못한다.\n \n       제2장 인\n         제1절 능력\n \n제3조(권리능력의 존속기간) 사람은 생존한 동안 권리와 의무의 주체가 된다.\n \n제4조(성년) 사람은 19세로 성년에 이르게 된다.\n[전문개정 2011. 3. 7.]\n \n제5조(미성년자의 능력) ①미성년자가 법률행위를 함에는 법정대리인의 동의를 얻어야 한다. 그러나 권리만을 얻거나\n의무만을 면하는 행위는 그러하지 아니하다.\n②전항의 규정에 위반한 행위는 취소할 수 있다.\n \n제6조(처분을 허락한 재산) 법정대리인이 범위를 정하여 처분을 허락한 재산은 미성년자가 임의로 처분할 수 있다.'),
 Document(metadata={'source': 'law1.pdf', 'page': 57}, page_content='제681조(수임인의 선관의무) 수임인은 위임의 본지에 따라 선량한 관리자의 주의로써 위임사무를 처리하여야 한다.\n \n제682조(복임권의 제한) ①수임인은 위임인의 승낙이나 부득이한 사유없이 제삼자로 하여금 자기에 갈음하여 위임사\n무를 처리하게 하지 못한다. <개정 2014. 12. 30.>\n②수임인이 전항의 규정에 의하여 제삼자에게 위임사무를 처리하게 한 경우에는 제121조, 제123조의 규정을 준용\n한다.\n \n제683조(수임인의 보고의무) 수임인은 위임인의 청구가 있는 때에는 위임사무의 처리상황을 보고하고 위임이 종료한\n때에는 지체없이 그 전말을 보고하여야 한다.'),
 Document(metadata={'source': 'law1.pdf', 'page': 0}, page_content='법제처                                                            1              

In [7]:
print(docs[0].metadata)

{'source': 'law1.pdf', 'page': 0}


In [8]:
from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="gemma2:9b", temperature=0, base_url="http://127.0.0.1:11434/") #http://127.0.0.1:11434

In [9]:
# #RAG
# from langchain.chains import RetrievalQA
# question = """신의성실의 원칙은 민법 몇 조이니?"""
# qa_chain = RetrievalQA.from_chain_type(llm,retriever=db.as_retriever())
# result = qa_chain({"query": question})
# print(result['result'])

In [11]:
#일반 쿼리
result = llm.predict("신의성실의 원칙에 대해 설명해줘")
print(result)

## 신의 성실의 원칙: 믿음과 행동의 조화

신의 성실은 **"신을 향한 진심 어린 헌신과 그 뜻에 따라 살아가는 것"**을 의미합니다. 이는 단순히 종교적인 의식이나 관습만을 따르는 것이 아니라, 마음속에서 신을 사랑하고 경외하며 그의 가르침을 실천하는 것을 포함합니다.

**핵심 원칙은 다음과 같습니다:**

* **진실된 믿음**: 신의 존재와 권능에 대한 확고한 믿음이 기반입니다. 이는 단순히 지식적인 이해를 넘어, 마음속에서 진정으로 신을 받아들이고 경외하는 태도를 의미합니다.
* **헌신과 순종**: 신의 뜻을 따르고 그의 가르침에 따라 살아가는 것을 최우선으로 여깁니다. 이는 자신의 의지와 욕망보다 신의 뜻을 존중하고, 그에 따라 행동하는 것을 의미합니다.
* **성실한 실천**: 믿음과 순종은 단순히 말로만 표현되는 것이 아니라, 일상생활에서 구체적으로 실천되어야 합니다. 이는 도덕적 행동, 타인을 향한 사랑, 그리고 신의 가르침에 따라 살아가는 모습으로 나타납니다.
* **끊임없는 노력**: 신의 성실은 단기간에 완성되는 것이 아니라, 꾸준한 노력과 자기 성찰을 통해 발전하는 과정입니다. 자신의 부족함을 인지하고, 끊임없이 개선하려는 의지를 가지고 살아가야 합니다.

**신의 성실은 단순히 종교적인 의미를 넘어, 인간으로서 진정한 행복과 만족을 찾기 위한 길입니다.** 신을 향한 진심 어린 사랑과 그의 가르침에 따라 살아가는 것은 우리 내면의 평화와 조화를 가져다주고, 세상에 대한 더 나은 이해와 관점을 제공합니다.



